In [15]:
#问题一附件1语音业务数据集处理代码
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from plotnine import *
# from joypy import joyplot
from sklearn.feature_selection import SelectKBest
# from minepy import MINE

In [16]:
#运行配置参数中的字体
plt.rcParams['font.sans-serif'] = ['STSong']
#运行配置参数总的轴(axes)正常显示正负号(minus)
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
#读取附件1数据
data_1 = pd.read_excel('./data/附件1语音业务用户满意度数据.xlsx')
data_1
#计算每一列缺失值的占比
def missing (df):
    missing_number = df.isnull().sum().sort_values(ascending=False)
    missing_percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_values = pd.concat([missing_number,missing_percent],axis=1,keys=['缺失值数量','缺失值所占百分比'])
    return missing_values
 
missing(data_1)

In [ ]:
#删除冗余特征、无用特征
data_1 = data_1.drop('用户id', axis = 1)
data_1 = data_1.drop('用户描述', axis = 1)
data_1 = data_1.drop('用户描述.1', axis = 1)
data_1 = data_1.drop('重定向次数', axis = 1)
data_1 = data_1.drop('重定向驻留时长', axis = 1)

In [ ]:
#缺失值填补
#是否关怀用户和是否去过营业厅，都用否来填充
data_1['是否关怀用户'].fillna('否', inplace=True)
data_1['是否去过营业厅'].fillna('否', inplace=True)
# 选择用众数填充
data_1['是否4G网络客户（本地剔除物联网）'].fillna('是', inplace = True)
data_1['终端品牌'].value_counts()
data_1['终端品牌'].fillna('苹果', inplace = True)
data_1['终端品牌类型'].value_counts()
data_1['终端品牌类型'].fillna('A2223', inplace = True)
data_1['是否5G网络客户'].fillna('否', inplace = True)
data_1['是否实名登记用户'].fillna('否', inplace = True)
data_1['客户星级标识'].value_counts()
data_1['客户星级标识'].fillna('三星', inplace = True)
# 选择用0填充
data_1['外省流量占比'].fillna(0.0 , inplace = True)
data_1['当月欠费金额'].fillna(0.00, inplace = True)
data_1['前第3个月欠费金额'].fillna(0.00, inplace = True)

In [ ]:
 #检查是否还有缺失值
plt.rcParams['font.sans-serif'] = ['STSong']
import missingno as msn
msn.matrix(data_1, labels = True)
msn.bar(data_1)

In [ ]:
#数据集的填充替换
#首先是对【'居民小区', '办公室', '高校', '商业街', '地铁', '农村', '高铁'】进行替换，-1表示“不在”，其他值（2，3，4，5，6，7）表示“在”
cols = ['居民小区', '办公室', '高校', '商业街', '地铁', '农村', '高铁']
j = 0
for col in cols:
        j += 1
        data_1[col].replace(-1, '不在', inplace = True)
        data_1[col].replace(j, '在', inplace = True)
data_1

In [ ]:
#检查列名
data_1.columns

In [ ]:
#“其他，请注明”和“其他，请注明.1”也需要进行替换，98表示“有”，-1表示没有
data_1['其他，请注明.1'].replace(98, '有', inplace = True)
data_1['其他，请注明.1'].replace(-1, '没有', inplace = True)
data_1['其他，请注明'].replace(98, '有', inplace = True)
data_1['其他，请注明'].replace(-1, '没有', inplace = True)
#“是否遇到网络问题”中，2表示否，1表示“是”
data_1['是否遇到过网络问题'].replace(2, '否', inplace = True)
data_1['是否遇到过网络问题'].replace(1, '是', inplace = True)
#['手机没有信号', '有信号无法拨通', '通话过程中突然中断', '通话中有杂音、听不清、断断续续', '串线', '通话过程中一方听不见']中，-1表示没有，1，2，3，4，5，6表示“有”
cols = ['手机没有信号', '有信号无法拨通', '通话过程中突然中断', '通话中有杂音、听不清、断断续续', '串线', '通话过程中一方听不见']
j = 0
for col in cols:
        j += 1
        data_1[col].replace(-1, '没有', inplace = True)
        data_1[col].replace(j, '有', inplace = True)
data_1

In [ ]:
#删除“终端品牌类型”这个无关特征
data_1 = data_1.drop('终端品牌类型', axis = 1)
data_1.to_excel('./data_1处理后的数据.xlsx')

In [ ]:
#根据“语音通话整体满意度”进行编码
import category_encoders as ce
df_1 = data_1.copy()
target = df_1[['语音通话整体满意度']]
train = df_1.drop(['语音通话整体满意度', '网络覆盖与信号强度', '语音通话清晰度', '语音通话稳定性'], axis = 1)
cbe_encoder = ce.cat_boost.CatBoostEncoder()
cbe_encoder.fit(train, target)
data = pd.DataFrame(cbe_encoder.transform(train))
data

In [ ]:
data['客户星级标识'].value_counts()

In [ ]:
#随机森林进行特征重要性提取
from sklearn.ensemble import RandomForestClassifier
X, y = data, df_1.iloc[:, 0]
feat_labels = df_1.columns[4 : ]
forest = RandomForestClassifier(n_estimators = 300 ,random_state = 10)
forest.fit(X, y)
importances = pd.Series(forest.feature_importances_, index = feat_labels).sort_values(ascending = False)
importances

In [ ]:
#语音通话整体满意度
#互信息
def mic(x, y):
    m = MINE()
    m.compute_score(x, y)
    return (m.mic(), 0.5)
index_4 = np.array(data.columns)
lst = []
for i in index_4:
    l = mic(data[i], df_1['语音通话整体满意度'])[0]
    lst.append(l)
w1 = pd.Series(pd.Series(lst).values, index = index_4).sort_values(ascending = False)
w1

In [ ]:
index = np.array(w1.index)
MIC = pd.DataFrame()
MIC['变量'] = index
MIC['互信息值'] = w1.values

In [ ]:
plt.rcParams['font.sans-serif'] = ['STSong']
plt.rcParams['figure.dpi'] = 1000
g = sns.catplot(data = MIC.iloc[: 20, :], x = '互信息值', y = '变量', kind = 'bar', height = 8, aspect = 1.875)
plt.title('语音通话整体满意度的互信息值', font={'family' : 'STSong', 'size' : 14})
plt.ylabel('变量', font={'family':'STSong', 'size':14})
plt.xlabel('互信息值', font={'family':'STSong', 'size':14})
df = MIC.iloc[: 20, :]
C = np.array(df['互信息值'])
for a, b, c in zip(df['互信息值'], range(20), C):
	plt.text(a + 0.0001, b, '%.3f'%c, fontsize = 14)

In [ ]:
#使用GBDT算法提取特征重要性
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators = 300, random_state = 10)
clf.fit(X, y)
var_importance = pd.Series(clf.feature_importances_, index = feat_labels).sort_values(ascending = False)
var_importance

In [ ]:
index = np.array(var_importance.index)
feature_importance = pd.DataFrame()
feature_importance['feature_names'] = index
feature_importance['feature_values'] = var_importance.values
feature_importance

In [ ]:
plt.rcParams['font.sans-serif'] = ['STSong']
plt.rcParams['figure.dpi'] = 1000
g = sns.catplot(data = feature_importance.iloc[: 20, :], x = 'feature_values', y = 'feature_names', kind = 'bar', height = 8, aspect = 1.875)
plt.title('语音通话整体满意度', font={'family' : 'STSong', 'size' : 14})
plt.ylabel('feature_names', font={'family':'STSong', 'size':14})
plt.xlabel('feature_values', font={'family':'STSong', 'size':14})
df = feature_importance.iloc[: 20, :]
C = np.array(df['feature_values'])
for a, b, c in zip(df['feature_values'], range(20), C):
	plt.text(a + 0.0001, b, '%.3f'%c, fontsize = 14)

In [ ]:
#网络覆盖与信号强度
#互信息
df_2 = data_1.copy()
target = df_2[['网络覆盖与信号强度']]
train = df_1.drop(['语音通话整体满意度', '网络覆盖与信号强度', '语音通话清晰度', '语音通话稳定性'], axis = 1)
cbe_encoder = ce.cat_boost.CatBoostEncoder()
cbe_encoder.fit(train, target)
data2 = pd.DataFrame(cbe_encoder.transform(train))
data2

In [ ]:
def mic(x, y):
    m = MINE()
    m.compute_score(x, y)
    return (m.mic(), 0.5)
 
index_4 = np.array(data2.columns)
lst = []
for i in index_4:
    l = mic(data2[i], df_2['网络覆盖与信号强度'])[0]
    lst.append(l)
w2 = pd.Series(pd.Series(lst).values, index = index_4).sort_values(ascending = False)
w2

In [ ]:
index = np.array(w2.index)
MIC2 = pd.DataFrame()
MIC2['变量'] = index
MIC2['互信息值'] = w2.values
plt.rcParams['font.sans-serif'] = ['STSong']
plt.rcParams['figure.dpi'] = 1000
g = sns.catplot(data = MIC2.iloc[: 20, :], x = '互信息值', y = '变量', kind = 'bar', height = 8, aspect = 1.875)
plt.title('网络覆盖与信号强度的互信息值', font={'family' : 'STSong', 'size' : 14})
plt.ylabel('变量', font={'family':'STSong', 'size':14})
plt.xlabel('互信息值', font={'family':'STSong', 'size':14})
df = MIC2.iloc[: 20, :]
C = np.array(df['互信息值'])
for a, b, c in zip(df['互信息值'], range(20), C):
	plt.text(a + 0.0001, b, '%.3f'%c, fontsize = 14)

In [ ]:
#使用GBDT算法提取特征重要性
X, y = data2, df_2.iloc[:, 1]
feat_labels = df_2.columns[4 : ]
clf2 = GradientBoostingClassifier(n_estimators = 300, random_state = 10)
clf2.fit(X, y)
var_importance2 = pd.Series(clf2.feature_importances_, index = feat_labels).sort_values(ascending = False)
var_importance2

In [ ]:
index2 = np.array(var_importance2.index)
feature_importance2 = pd.DataFrame()
feature_importance2['feature_names'] = index2
feature_importance2['feature_values'] = var_importance2.values
feature_importance2

In [ ]:
plt.rcParams['font.sans-serif'] = ['STSong']
plt.rcParams['figure.dpi'] = 1000
g = sns.catplot(data = feature_importance2.iloc[: 20, :], x = 'feature_values', y = 'feature_names', kind = 'bar', height = 8, aspect = 1.875)
plt.title('网络覆盖与信号强度', font={'family' : 'STSong', 'size' : 14})
plt.ylabel('feature_names', font={'family':'STSong', 'size':14})
plt.xlabel('feature_values', font={'family':'STSong', 'size':14})
df = feature_importance2.iloc[: 20, :]
C = np.array(df['feature_values'])
for a, b, c in zip(df['feature_values'], range(20), C):
	plt.text(a + 0.0001, b, '%.3f'%c, fontsize = 14)

In [ ]:
#语音通话清晰度
#互信息
df_3 = data_1.copy()
target = df_3[['语音通话清晰度']]
train = df_1.drop(['语音通话整体满意度', '网络覆盖与信号强度', '语音通话清晰度', '语音通话稳定性'], axis = 1)
cbe_encoder = ce.cat_boost.CatBoostEncoder()
cbe_encoder.fit(train, target)
data3 = pd.DataFrame(cbe_encoder.transform(train))

In [ ]:
def mic(x, y):
    m = MINE()
    m.compute_score(x, y)
    return (m.mic(), 0.5)
index_4 = np.array(data3.columns)
lst = []
for i in index_4:
    l = mic(data3[i], df_3['语音通话清晰度'])[0]
    lst.append(l)
w3 = pd.Series(pd.Series(lst).values, index = index_4).sort_values(ascending = False)
w3

In [ ]:
index = np.array(w3.index)
MIC3 = pd.DataFrame()
MIC3['变量'] = index
MIC3['互信息值'] = w3.values
plt.rcParams['font.sans-serif'] = ['STSong']
plt.rcParams['figure.dpi'] = 1000
g = sns.catplot(data = MIC3.iloc[: 20, :], x = '互信息值', y = '变量', kind = 'bar', height = 8, aspect = 1.875)
plt.title('语音通话清晰度的互信息值', font={'family' : 'STSong', 'size' : 14})
plt.ylabel('变量', font={'family':'STSong', 'size':14})
plt.xlabel('互信息值', font={'family':'STSong', 'size':14})
df = MIC3.iloc[: 20, :]
C = np.array(df['互信息值'])
for a, b, c in zip(df['互信息值'], range(20), C):
	plt.text(a + 0.0001, b, '%.3f'%c, fontsize = 14)

In [ ]:
#使用GBDT算法提取特征重要性
X, y = data3, df_3.iloc[:, 2]
feat_labels = df_3.columns[4 : ]
clf3 = GradientBoostingClassifier(n_estimators = 300, random_state = 10)
clf3.fit(X, y)
var_importance3 = pd.Series(clf3.feature_importances_, index = feat_labels).sort_values(ascending = False)
var_importance3

In [ ]:
index3 = np.array(var_importance3.index)
feature_importance3 = pd.DataFrame()
feature_importance3['feature_names'] = index3
feature_importance3['feature_values'] = var_importance3.values

In [ ]:
plt.rcParams['font.sans-serif'] = ['STSong']
plt.rcParams['figure.dpi'] = 1000
g = sns.catplot(data = feature_importance3.iloc[: 20, :], x = 'feature_values', y = 'feature_names', kind = 'bar', height = 8, aspect = 1.875)
plt.title('语音通话清晰度', font={'family' : 'STSong', 'size' : 14})
plt.ylabel('feature_names', font={'family':'STSong', 'size':14})
plt.xlabel('feature_values', font={'family':'STSong', 'size':14})
df = feature_importance3.iloc[: 20, :]
C = np.array(df['feature_values'])
for a, b, c in zip(df['feature_values'], range(20), C):
	plt.text(a + 0.0001, b, '%.3f'%c, fontsize = 14)

In [ ]:
#语音通话稳定性
#互信息
df_4 = data_1.copy()
target = df_4[['语音通话稳定性']]
train = df_1.drop(['语音通话整体满意度', '网络覆盖与信号强度', '语音通话清晰度', '语音通话稳定性'], axis = 1)
cbe_encoder = ce.cat_boost.CatBoostEncoder()
cbe_encoder.fit(train, target)
data4 = pd.DataFrame(cbe_encoder.transform(train))

In [ ]:
def mic(x, y):
    m = MINE()
    m.compute_score(x, y)
    return (m.mic(), 0.5)
index_4 = np.array(data4.columns)
lst = []
for i in index_4:
    l = mic(data4[i], df_4['语音通话稳定性'])[0]
    lst.append(l)
w4 = pd.Series(pd.Series(lst).values, index = index_4).sort_values(ascending = False)
w4

In [ ]:
index = np.array(w4.index)
MIC4 = pd.DataFrame()
MIC4['变量'] = index
MIC4['互信息值'] = w4.values
plt.rcParams['font.sans-serif'] = ['STSong']
plt.rcParams['figure.dpi'] = 1000
g = sns.catplot(data = MIC4.iloc[: 20, :], x = '互信息值', y = '变量', kind = 'bar', height = 8, aspect = 1.875)
plt.title('语音通话清晰度的互信息值', font={'family' : 'STSong', 'size' : 14})
plt.ylabel('变量', font={'family':'STSong', 'size':14})
plt.xlabel('互信息值', font={'family':'STSong', 'size':14})
df = MIC4.iloc[: 20, :]
C = np.array(df['互信息值'])
for a, b, c in zip(df['互信息值'], range(20), C):
	plt.text(a + 0.0001, b, '%.3f'%c, fontsize = 14)

In [ ]:
#使用GBDT算法提取特征重要性
X, y = data4, df_4.iloc[:, 2]
feat_labels = df_4.columns[4 : ]
clf4 = GradientBoostingClassifier(n_estimators = 300, random_state = 10)
clf4.fit(X, y)
var_importance4 = pd.Series(clf4.feature_importances_, index = feat_labels).sort_values(ascending = False)
var_importance4

In [ ]:
index4 = np.array(var_importance4.index)
feature_importance4 = pd.DataFrame()
feature_importance4['feature_names'] = index4
feature_importance4['feature_values'] = var_importance4.values

In [ ]:
plt.rcParams['font.sans-serif'] = ['STSong']
plt.rcParams['figure.dpi'] = 1000
g = sns.catplot(data = feature_importance4.iloc[: 20, :], x = 'feature_values', y = 'feature_names', kind = 'bar', height = 8, aspect = 1.875)
plt.title('语音通话稳定性', font={'family' : 'STSong', 'size' : 14})
plt.ylabel('feature_names', font={'family':'STSong', 'size':14})
plt.xlabel('feature_values', font={'family':'STSong', 'size':14})
df = feature_importance4.iloc[: 20, :]
C = np.array(df['feature_values'])
for a, b, c in zip(df['feature_values'], range(20), C):
	plt.text(a + 0.0001, b, '%.3f'%c, fontsize = 14)

In [ ]:
#相关系数图
data_1.iloc[:,1:5].corr()
# Plot
def plot_corr(y_X):
    plt.figure(figsize=(18,3.4), dpi= 1000)
    titles = ['pearson','kendall','spearman']
    for i,title in enumerate(titles):
        plt.subplot(1, 3, i+1)
        sns.heatmap(y_X.corr(title),  # 相关系数矩阵
                    xticklabels=y_X.corr().columns, # 横坐标标签
                    yticklabels=y_X.corr().columns, # 横坐标标签
                    cmap='copper', 
                    center=0, 
                    square=True,
                    annot=True
                    ) 
        
        plt.title(label=title+'相关系数', fontsize=12)
        plt.xticks(fontsize=10,
                   rotation=45,# 旋转
                   horizontalalignment='right') # 刻度的相对位置
        plt.yticks(fontsize=10,rotation=0,)
    plt.show()
 
plot_corr(data_1.iloc[ :,1:5])

In [ ]:
#热力图
data1 = data.loc[:,index[:20]]
plt.figure(figsize = (15, 15), dpi = 1000)
sns.heatmap(data1.corr(), cmap='YlGnBu')

In [ ]:
#问题一量化分析图
#是否遇到过网络问题
plt.figure(figsize = (15, 15), dpi = 1000)
plt.subplot(4, 2, 1)
sns.countplot(data = data_1, x = '语音通话整体满意度', hue = '是否遇到过网络问题')
df1 = data_1.groupby(['语音通话整体满意度', '是否遇到过网络问题']).size().reset_index(name='Count')
no1 = [15,3,3,0,14,12,27,122,270,2107]
yes1 = (df1.loc[df1['是否遇到过网络问题'] == '是']).loc[:,'Count']
x1 = list(range(len(yes1)))
for a,b in zip(x1, no1): ##控制标签位置
    plt.text(a+0.18, b+0.1, '%.0f'%b, ha = 'center', va = 'bottom', fontsize = 8)
for a,b in zip(x1, yes1):
    plt.text(a-0.2, b+0.1, '%.0f'%b, ha = 'center', va = 'bottom', fontsize = 8)
plt.subplot(4, 2, 2)
sns.countplot(data = data_1, x = '网络覆盖与信号强度', hue = '是否遇到过网络问题')
df2 = data_1.groupby(['网络覆盖与信号强度', '是否遇到过网络问题']).size().reset_index(name='Count')
no2 = (df2.loc[df2['是否遇到过网络问题'] == '否']).loc[:,'Count']
yes2 = (df2.loc[df2['是否遇到过网络问题'] == '是']).loc[:,'Count']
x2 = list(range(len(yes2)))
for a,b in zip(x2, no2): ##控制标签位置
    plt.text(a+0.18,b+0.1,'%.0f'%b,ha = 'center',va = 'bottom', fontsize = 8)
for a,b in zip(x2, yes2):
    plt.text(a-0.2, b+0.1, '%.0f'%b, ha = 'center', va = 'bottom', fontsize = 8)

In [ ]:
plt.subplot(4, 2, 3)
sns.countplot(data = data_1, x = '语音通话清晰度', hue = '是否遇到过网络问题')
df3 = data_1.groupby(['语音通话清晰度', '是否遇到过网络问题']).size().reset_index(name='Count')
no3 = (df3.loc[df3['是否遇到过网络问题'] == '否']).loc[:,'Count']
yes3 = (df3.loc[df3['是否遇到过网络问题'] == '是']).loc[:,'Count']
x3 = list(range(len(yes3)))
for a,b in zip(x3, no3): ##控制标签位置
    plt.text(a+0.18, b+0.1, '%.0f'%b, ha = 'center', va = 'bottom', fontsize = 8)
for a,b in zip(x3, yes3):
    plt.text(a-0.2, b+0.1, '%.0f'%b, ha = 'center', va = 'bottom', fontsize = 8)
    
    
plt.subplot(4, 2, 4)
sns.countplot(data = data_1, x = '语音通话稳定性', hue = '是否遇到过网络问题')
df4 = data_1.groupby(['语音通话稳定性', '是否遇到过网络问题']).size().reset_index(name='Count')
no4 = (df4.loc[df4['是否遇到过网络问题'] == '否']).loc[:,'Count']
yes4 = (df4.loc[df4['是否遇到过网络问题'] == '是']).loc[:,'Count']
x4 = list(range(len(yes4)))
for a,b in zip(x4, no4): ##控制标签位置
    plt.text(a+0.18, b+0.1, '%.0f'%b, ha = 'center', va = 'bottom', fontsize = 8)
for a,b in zip(x4, yes4):
    plt.text(a-0.2, b+0.1, '%.0f'%b, ha = 'center', va = 'bottom', fontsize = 8)

In [ ]:
#GPRS总流量（GB）
max1 = (data_1.loc[data_1.groupby('语音通话整体满意度')['GPRS总流量（KB）'].idxmax(), :].reset_index())['GPRS总流量（KB）']/(1024*1024)
score = pd.DataFrame(np.arange(1, 11))
df1 = pd.concat([score, max1], axis = 1)
df1.columns = ['语音通话整体满意度', 'GPRS_max(GB)']
 
max2 = (data_1.loc[data_1.groupby('网络覆盖与信号强度')['GPRS总流量（KB）'].idxmax(), :].reset_index())['GPRS总流量（KB）']/(1024*1024)
score = pd.DataFrame(np.arange(1, 11))
df2 = pd.concat([score, max2], axis = 1)
df2.columns = ['网络覆盖与信号强度', 'GPRS_max(GB)']
 
max3 = (data_1.loc[data_1.groupby('语音通话清晰度')['GPRS总流量（KB）'].idxmax(), :].reset_index())['GPRS总流量（KB）']/(1024*1024)
score = pd.DataFrame(np.arange(1, 11))
df3 = pd.concat([score, max3], axis = 1)
df3.columns = ['语音通话清晰度', 'GPRS_max(GB)']
 
max4 = ((data_1.loc[data_1.groupby('语音通话稳定性')['GPRS总流量（KB）'].idxmax(), :].reset_index())['GPRS总流量（KB）'])/(1024*1024)
score = pd.DataFrame(np.arange(1, 11))
df4 = pd.concat([score, max4], axis = 1)
df4.columns = ['语音通话稳定性', 'GPRS_max(GB)']
 
newdf = pd.concat([df1['语音通话整体满意度'], df1['GPRS_max(GB)'], df2['GPRS_max(GB)'], df3['GPRS_max(GB)'], df4['GPRS_max(GB)']], axis=1)
newdf.columns = ['打分', '语音通话整体满意度', '网络覆盖与信号强度', '语音通话清晰度', '语音通话稳定性']
newdf

In [ ]:
#当月ARPU
max1 = (data_1.loc[data_1.groupby('语音通话整体满意度')['当月ARPU'].idxmax(), :].reset_index())['当月ARPU']
min1 = (data_1.loc[data_1.groupby('语音通话整体满意度')['当月ARPU'].idxmin(), :].reset_index())['当月ARPU']
score = pd.DataFrame(np.arange(1, 11))
df1 = pd.concat([score,min1,max1], axis = 1)
df1.columns = ['语音通话整体满意度','当月ARPU_min','当月ARPU_max']
 
max2 = (data_1.loc[data_1.groupby('网络覆盖与信号强度')['当月ARPU'].idxmax(), :].reset_index())['当月ARPU']
min2 = (data_1.loc[data_1.groupby('网络覆盖与信号强度')['当月ARPU'].idxmin(), :].reset_index())['当月ARPU']
score = pd.DataFrame(np.arange(1, 11))
df2 = pd.concat([score,min2,max2], axis = 1)
df2.columns = ['网络覆盖与信号强度','当月ARPU_min','当月ARPU_max']
 
max3 = (data_1.loc[data_1.groupby('语音通话清晰度')['当月ARPU'].idxmax(), :].reset_index())['当月ARPU']
min3 = (data_1.loc[data_1.groupby('语音通话清晰度')['当月ARPU'].idxmin(), :].reset_index())['当月ARPU']
score = pd.DataFrame(np.arange(1, 11))
df3 = pd.concat([score,min3,max3], axis = 1)
df3.columns = ['语音通话清晰度','当月ARPU_min','当月ARPU_max']
 
max4 = (data_1.loc[data_1.groupby('语音通话稳定性')['当月ARPU'].idxmax(), :].reset_index())['当月ARPU']
min4 = (data_1.loc[data_1.groupby('语音通话稳定性')['当月ARPU'].idxmin(), :].reset_index())['当月ARPU']
score = pd.DataFrame(np.arange(1, 11))
df4 = pd.concat([score,min4,max4], axis = 1)
df4.columns = ['语音通话稳定性','当月ARPU_min','当月ARPU_max']
 
newdf = pd.concat([df1['语音通话整体满意度'], df1['当月ARPU_max'], df2['当月ARPU_max'], df3['当月ARPU_max'], df4['当月ARPU_max']], axis=1)
newdf.columns = ['打分', '语音通话整体满意度', '网络覆盖与信号强度', '语音通话清晰度', '语音通话稳定性']
newdf